In [290]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re
import pandas as pd
import math
import nltk

In [187]:
discography = pd.read_csv("../data/beatles_full.csv")

In [188]:
discography

,album,track,popularity,album_popularity,album_tracks,avg_album_popularity,lyrics
0,Please Please Me,I Saw Her Standing There,70,819,14,58.50,"One, two, three, four!. Well, she was just sev..."
1,Please Please Me,Misery,53,819,14,58.50,"The world is treating me bad, misery. . I'm th..."
2,Please Please Me,Anna,61,819,14,58.50,"Anna. You come and ask me, girl. To set you fr..."
3,Please Please Me,Chains,50,819,14,58.50,"Chains, my baby's got me locked up in chains. ..."
4,Please Please Me,Boys,51,819,14,58.50,I been told when a boy kiss a girl. Take a tri...
...,...,...,...,...,...,...,...
188,Let It Be,I've Got A Feeling,64,776,12,64.67,"I've got a feeling, a feeling deep inside. Oh ..."
189,Let It Be,One After 909,60,776,12,64.67,My baby said she's traveling on the one after ...
190,Let It Be,The Long And Winding Road,67,776,12,64.67,The long and winding road that leads to your d...
191,Let It Be,For You Blue,59,776,12,64.67,". . Because you're sweet and lovely, girl, I l..."


# SENTIMENT

In [189]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [190]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [191]:
discography['sentiment'] = discography['lyrics'].apply(lambda x: sentiment_score(x[:512]))

In [192]:
#apresentar em wordclouds nome das músicas mais felizes/tristes/oks

In [193]:
tmp_series = discography[['album', 'sentiment']].groupby(by='album').sum()['sentiment']

tmp_series = tmp_series.reindex(discography[['album', 'sentiment']].groupby(by='album').sum().index)

tmp_series

album
A Hard Day's Night                       40
Abbey Road                               50
Beatles For Sale                         36
Help!                                    53
Let It Be                                42
Magical Mystery Tour                     41
Please Please Me                         58
Revolver                                 43
Rubber Soul                              35
Sgt. Pepper's Lonely Hearts Club Band    54
The Beatles                              93
With The Beatles                         43
Yellow Submarine                         40
Name: sentiment, dtype: int64

In [194]:
discography['album_sentiment'] = discography['album'].map(tmp_series)
discography['avg_album_sentiment'] = round(discography['album_sentiment']/discography['album_tracks'],4)

discography

,album,track,popularity,album_popularity,album_tracks,avg_album_popularity,lyrics,sentiment,album_sentiment,avg_album_sentiment
0,Please Please Me,I Saw Her Standing There,70,819,14,58.50,"One, two, three, four!. Well, she was just sev...",5,58,4.1429
1,Please Please Me,Misery,53,819,14,58.50,"The world is treating me bad, misery. . I'm th...",1,58,4.1429
2,Please Please Me,Anna,61,819,14,58.50,"Anna. You come and ask me, girl. To set you fr...",5,58,4.1429
3,Please Please Me,Chains,50,819,14,58.50,"Chains, my baby's got me locked up in chains. ...",1,58,4.1429
4,Please Please Me,Boys,51,819,14,58.50,I been told when a boy kiss a girl. Take a tri...,5,58,4.1429
...,...,...,...,...,...,...,...,...,...,...
188,Let It Be,I've Got A Feeling,64,776,12,64.67,"I've got a feeling, a feeling deep inside. Oh ...",1,42,3.5000
189,Let It Be,One After 909,60,776,12,64.67,My baby said she's traveling on the one after ...,1,42,3.5000
190,Let It Be,The Long And Winding Road,67,776,12,64.67,The long and winding road that leads to your d...,5,42,3.5000
191,Let It Be,For You Blue,59,776,12,64.67,". . Because you're sweet and lovely, girl, I l...",5,42,3.5000


In [195]:
#'linha' de triste a feliz (coração partido até coração)
discography[['album', 'avg_album_sentiment']].drop_duplicates().sort_values('avg_album_sentiment')

,album,avg_album_sentiment
69,Rubber Soul,2.5000
41,Beatles For Sale,2.5714
164,Abbey Road,2.9412
14,With The Beatles,3.0714
83,Revolver,3.0714
28,A Hard Day's Night,3.0769
151,Yellow Submarine,3.0769
121,The Beatles,3.1000
181,Let It Be,3.5000
110,Magical Mystery Tour,3.7273


# LEXICAL RICHNESS

In [199]:
def cttr(lyrics):
    tokens = word_tokenize(lyrics)
    return round(len(set(tokens))/math.sqrt(2 * len(tokens)), 2)

In [200]:
discography['cttr'] = discography['lyrics'].apply(lambda x: cttr(x))

In [201]:
tmp_series = discography[['album', 'cttr']].groupby(by='album').sum()['cttr']

tmp_series = tmp_series.reindex(discography[['album', 'cttr']].groupby(by='album').sum().index)

discography['album_cttr'] = discography['album'].map(tmp_series)
discography['avg_album_cttr'] = round(discography['album_cttr']/discography['album_tracks'],4)

discography

,album,track,popularity,album_popularity,album_tracks,avg_album_popularity,lyrics,sentiment,album_sentiment,avg_album_sentiment,cttr,album_cttr,avg_album_cttr
0,Please Please Me,I Saw Her Standing There,70,819,14,58.50,"One, two, three, four!. Well, she was just sev...",5,58,4.1429,3.63,40.26,2.8757
1,Please Please Me,Misery,53,819,14,58.50,"The world is treating me bad, misery. . I'm th...",1,58,4.1429,3.77,40.26,2.8757
2,Please Please Me,Anna,61,819,14,58.50,"Anna. You come and ask me, girl. To set you fr...",5,58,4.1429,3.26,40.26,2.8757
3,Please Please Me,Chains,50,819,14,58.50,"Chains, my baby's got me locked up in chains. ...",1,58,4.1429,3.37,40.26,2.8757
4,Please Please Me,Boys,51,819,14,58.50,I been told when a boy kiss a girl. Take a tri...,5,58,4.1429,2.61,40.26,2.8757
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Let It Be,I've Got A Feeling,64,776,12,64.67,"I've got a feeling, a feeling deep inside. Oh ...",1,42,3.5000,3.25,38.55,3.2125
189,Let It Be,One After 909,60,776,12,64.67,My baby said she's traveling on the one after ...,1,42,3.5000,2.88,38.55,3.2125
190,Let It Be,The Long And Winding Road,67,776,12,64.67,The long and winding road that leads to your d...,5,42,3.5000,3.86,38.55,3.2125
191,Let It Be,For You Blue,59,776,12,64.67,". . Because you're sweet and lovely, girl, I l...",5,42,3.5000,2.63,38.55,3.2125


In [202]:
discography[['album', 'avg_album_cttr']].drop_duplicates().sort_values('avg_album_cttr')

,album,avg_album_cttr
14,With The Beatles,2.8021
41,Beatles For Sale,2.8457
0,Please Please Me,2.8757
110,Magical Mystery Tour,2.9500
181,Let It Be,3.2125
164,Abbey Road,3.2612
28,A Hard Day's Night,3.2677
55,Help!,3.3071
83,Revolver,3.4700
69,Rubber Soul,3.4993


## EMOTIONS

In [280]:
def emotions(lyrics):
    emotion = NRCLex(lyrics)
    emotion_freqs = emotion.affect_frequencies
    
    emotions_list = []
    
    emotions_list.append(emotion_freqs.get('fear'))
    emotions_list.append(emotion_freqs.get('anger'))
    emotions_list.append(emotion_freqs.get('trust'))
    emotions_list.append(emotion_freqs.get('surprise'))
    emotions_list.append(emotion_freqs.get('sadness'))
    emotions_list.append(emotion_freqs.get('disgust'))
    emotions_list.append(emotion_freqs.get('joy'))
    emotions_list.append(emotion_freqs.get('anticipation'))
    
    return emotions_list

In [286]:
discography['fear'], discography['anger'], discography['trust'], discography['surprise'], discography['sadness'], discography['disgust'], discography['joy'], discography['anticipation']= zip(*discography['lyrics'].apply(emotions))

In [294]:
discography.sort_values('anger')

,album,track,popularity,album_popularity,album_tracks,avg_album_popularity,lyrics,sentiment,album_sentiment,avg_album_sentiment,...,album_cttr,avg_album_cttr,anger,trust,surprise,sadness,disgust,joy,anticipation,fear
0,Please Please Me,I Saw Her Standing There,70,819,14,58.50,"One, two, three, four!. Well, she was just sev...",5,58,4.1429,...,40.26,2.8757,0.000000,0.121212,0.000000,0.090909,0.000000,0.212121,0.090909,0.000000
124,The Beatles,"ObLaDi, ObLaDa",72,1741,30,58.03,not found,1,93,3.1000,...,105.34,3.5113,0.000000,0.333333,0.000000,0.000000,0.000000,0.333333,None,0.000000
123,The Beatles,Glass Onion,58,1741,30,58.03,I told you 'bout Strawberry Fields. You know t...,1,93,3.1000,...,105.34,3.5113,0.000000,0.285714,0.000000,0.000000,0.142857,0.000000,0.142857,0.000000
122,The Beatles,Dear Prudence,64,1741,30,58.03,Dear Prudence. Won't you come out to play?. De...,5,93,3.1000,...,105.34,3.5113,0.000000,0.142857,0.142857,0.071429,0.000000,0.238095,0.119048,0.000000
120,Magical Mystery Tour,All You Need Is Love,67,674,11,61.27,"Love, love, love. Love, love, love. Love, love...",5,41,3.7273,...,32.45,2.9500,0.000000,0.029851,0.000000,0.014925,0.000000,0.447761,0.029851,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Let It Be,Let It Be,79,776,12,64.67,"When I find myself in times of trouble, Mother...",5,42,3.5000,...,38.55,3.2125,0.200000,0.175000,0.025000,0.075000,0.000000,0.025000,0.025,0.025000
41,Beatles For Sale,No Reply,60,720,14,51.43,This happened once before when I came to your ...,1,36,2.5714,...,39.84,2.8457,0.222222,0.000000,0.000000,0.222222,0.222222,0.000000,None,0.000000
182,Let It Be,Dig A Pony,62,776,12,64.67,"I dig a pony. Well, you can celebrate anything...",5,42,3.5000,...,38.55,3.2125,0.222222,0.000000,0.000000,0.000000,0.111111,0.000000,None,0.000000
128,The Beatles,Happiness Is A Warm Gun,61,1741,30,58.03,She's not a girl who misses much. Do do do do ...,3,93,3.1000,...,105.34,3.5113,0.279070,0.000000,0.000000,0.000000,0.000000,0.023256,0.023256,0.302326


In [397]:
emotions = discography.set_index(discography.track)
emotions = emotions[['fear', 'anger', 'trust', 'surprise', 'sadness', 'disgust', 'joy', 'anticipation']]

list(emotions.loc['I Saw Her Standing There'])

[0.0,
 0.0,
 0.12121212121212122,
 0.0,
 0.09090909090909091,
 0.0,
 0.21212121212121213,
 0.09090909090909091]

In [381]:
import plotly.graph_objects as go
import plotly.offline as pyo

In [388]:
def radar(emotions):

    categories = ['Fear', 'Anger', 'Trust', 'Surprise', 'Sadness', 'Disgust', 'Joy', 'Anticipation']
    categories = [*categories, categories[0]]

    restaurant_1 = emotions
    restaurant_1 = [*restaurant_1, restaurant_1[0]]

    fig = go.Figure(
        data=[
            go.Scatterpolar(r=restaurant_1, theta=categories, fill='toself', name='Restaurant 1')
        ],
        layout=go.Layout(
            title=go.layout.Title(text='Restaurant comparison'),
            polar={'radialaxis': {'visible': True}},
            showlegend=True
        )
    )

    pyo.plot(fig)

In [431]:
a = list(emotions.loc['Glass Onion'])
print(a)
norm = [(i * 10) + 0.2 for i in a]
print(norm)
radar(norm)

[0.0, 0.0, 0.2857142857142857, 0.0, 0.0, 0.14285714285714285, 0.0, 0.14285714285714285]
[0.2, 0.2, 3.057142857142857, 0.2, 0.2, 1.6285714285714283, 0.2, 1.6285714285714283]
